In [19]:
from Methods import *
def generate_X_AB(m, n, r, prop, random_state=1):
    np.random.seed(random_state)
    random.seed(random_state)
    true_A = np.random.normal(size=(m, r))
    true_B = np.random.normal(size=(n, r))
    X = true_A.dot(true_B.T).flatten()
    nan_pos = random.sample(range(m*n), int(m*n*prop))
    X[nan_pos] = np.NAN
    return X.reshape(m, n), true_A, true_B

X, true_A, true_B = generate_X_AB(m=100, n=100, r=15, prop=0.6, random_state=1)
X += np.random.normal(scale=1, size=(100, 100))
Lambda = 25
Rank = 100
print("Lambda:", Lambda)
print("Rank:",Rank)

def calculate_test_error(X, true_A, true_B, X_est):
    Omega_c = np.isnan(X)
    return np.linalg.norm((true_A.dot(true_B.T) - X_est)[Omega_c])**2 / \
        np.linalg.norm((true_A.dot(true_B.T))[Omega_c])**2


def calculate_train_error(X, X_est):
    Omega_c = np.isnan(X)
    Omega = np.array([not Omega_c[i, j] for i in range(
        Omega_c.shape[0]) for j in range(Omega_c.shape[1])]).reshape(Omega_c.shape)
    return np.linalg.norm((X - X_est)[Omega])**2 / \
        np.linalg.norm(X[Omega])**2

SoftImputer_SVD = SoftImpute_SVD_dense(
            X, Lambda=Lambda, r=Rank, sc='variable',warm_start=None)
SoftImputer_SVD.matrix_completion(
            rvar_eps=1e-9, robj_eps=1e-9, max_iter=3000)

Lambda: 25
Rank: 100
iteration: 41
relative objective and variable change: 1.330483731400579e-10 7.40040897058489e-10


In [20]:
def post_processing(X,U,V,r):
    Omega_c = np.isnan(X)
    Omega = np.array([not Omega_c[i, j] for i in range(
        Omega_c.shape[0]) for j in range(Omega_c.shape[1])]).reshape(Omega_c.shape)
    x0 = X[Omega].reshape(-1,1)
    A = np.zeros((x0.shape[0],r))
    for i in range(r):
        uvT=U[:,i].reshape(-1,1).dot(V[:,i].reshape(1,-1))
        A[:,i] = uvT[Omega]

    a = np.linalg.pinv(A).dot(x0)
    nega_cond = np.where(a<0)[0]
    a[nega_cond] *=-1
    U[:,nega_cond] *=-1
    return a,U
    
rank = np.where(SoftImputer_SVD.Dsq.diagonal()>=1e-10)[0].shape[0]
print(rank)
a,U = post_processing(X,SoftImputer_SVD.U,SoftImputer_SVD.V,rank)
print(calculate_test_error(X, true_A, true_B,SoftImputer_SVD.M))
U_r = U[:,:rank]
V_r = SoftImputer_SVD.V[:,:rank]
D_r = np.diag(a[:rank,0])
X_est=U_r.dot(D_r.dot(V_r.T))
print(a[:,0],SoftImputer_SVD.Dsq.diagonal())
print(calculate_test_error(X, true_A, true_B,X_est))

15
0.5785338609135566
[119.23698954 110.55569124  96.32036526  94.73782801  89.56002804
  85.03900628  81.25574769  78.53064091  61.94715067  58.31367343
  53.10496264  50.73858046  43.16475561  40.67336809  32.73840037] [7.69927402e+01 6.80792506e+01 5.81805502e+01 5.40720218e+01
 4.91889879e+01 4.54580860e+01 3.82338787e+01 3.72600766e+01
 2.38780437e+01 2.08190236e+01 1.45230095e+01 1.24279643e+01
 6.76973662e+00 1.66374080e+00 2.74485363e-09 7.69312016e-15
 7.69312016e-15 7.69312016e-15 7.69312016e-15 7.69312016e-15
 7.69312016e-15 7.69312016e-15 7.69312016e-15 7.69312016e-15
 7.69312016e-15 7.69312016e-15 7.69312016e-15 7.69312016e-15
 7.69312016e-15 7.69312016e-15 7.69312016e-15 7.69312016e-15
 7.69312016e-15 7.69312016e-15 7.69312016e-15 7.69312016e-15
 7.69312016e-15 7.69312016e-15 7.69312016e-15 7.69312016e-15
 7.69312016e-15 7.69312016e-15 7.69312016e-15 7.69312016e-15
 7.69312016e-15 7.69312016e-15 7.69312016e-15 7.69312016e-15
 7.69312016e-15 7.69312016e-15 7.69312016e-15 7